In [43]:
import pandas as pd
import numpy as np
import random
import time
import uuid
from datetime import datetime
from email.utils import parsedate_to_datetime

size = 25000
dates = range(int(time.mktime(( 2022, 1, 1, 0, 0, 0, 0, 0, 0))), int(time.mktime(( 2022, 1, 31, 0, 0, 0, 0, 0, 0)) ))
users_ids = [uuid.uuid4() for i in range(1000)]
users_advertiser_ids = [uuid.uuid4() for i in range((500))]
countries = {"FR":"France","GE":"Geramny","US":"United States","CA":"Canada","EN":"England","SP":"Spain","BR":"Brasil"}
browsers = {"345978365835376":"Chrome","8753685264337":"Mozilla Firefox","65784523663644":"Opera","2109367534582756":"Microsoft Edge","4534637355634":"Brave"}
data = pd.DataFrame()
data["Landing Page URL ID"] = [uuid.uuid4() for i in range(size)]
data["Referrer URL"] = [random.choice(['https://junglebike.fr/casque.html','https://www.amazon.fr/pulles/', 'https://www.cdiscount.com/', 'https://fr.aliexpress.com/', 'https://shopping.google.fr/']) for i in range(size)]
data["Impression ID"] = [random.choice(["AS","BP","AK","BS"])+str(random.choice(range(983456786862, 983456786862+200))) for i in range(size)]
data["Asset ID"] = [random.choice(range(1011566786, 1011566786+1000)) for i in range(size)]
data["Active View: Eligible Impressions"] = [random.choice(range(56863854982, 56863854982+2000)) for i in range(size)]
data["Active View: Measurable Impressions"] = [random.choice(range(7743646, 7743646+200)) for i in range(size)]
data["Active View: Viewable Impressions"] = [random.choice(range(4436462, 4436462+100)) for i in range(size)]
data["Activity ID"] = [random.choice(range(2398, 2398+50)) for i in range(size)]
data["Event Time"] = [parsedate_to_datetime(time.ctime(random.choice(dates))).replace(tzinfo=None).strftime("%Y-%m-%d %H:%M:%S") for i in range(size)]
data["Event Type"] = [random.choice(['VIEW', 'CLICK', 'CONVERSION', None]) for i in range(size)]
data["Event Sub-Type"] = [random.choice(['VIEW', 'CLICK', 'POSTVIEW','POSTCLICK', None]) for i in range(size)]

data["User ID"] = [random.choice(users_ids) for i in range(size)]
data["Advertiser ID"] = [random.choice(range(5678, 5678+235)) for i in range(size)]
data["Campaign ID"] = [random.choice(range(6768675387, 6768675387+100)) for i in range(size)]
data["Floodlight Configuration"] = [random.choice(range(1209785242, 1209785242+100)) for i in range(size)]
data["Ad ID"] = [random.choice(users_advertiser_ids) for i in range(size)]
data["Rendering ID"] = [random.choice(range(565726383, 565726383+50))  for i in range(size)]
data["Creative Version"] = [random.choice(range(67725, 67725+600)) for i in range(size)]
data["Site ID (DCM)"] = [random.choice(range(67725, 67725+600)) for i in range(size)]
data["Placement ID"] = [random.choice(range(565726383, 565726383+50)) for i in range(size)]
data["Country Code"] = random.choices(list(countries.keys()), weights=[0.15117245, 0.174, 0.25081, 0.116, 0.1256, 0.09, 0.08], k=size)
data["State/Region"] = data["Country Code"].apply(lambda x: countries[x])
data["Browser/Platform ID"] = random.choices(list(browsers.keys()), weights=[0.4817245, 0.24, 0.06081, 0.156, 0.056], k=size)
#data["Browser/Platform Version"] = data["Browser/Platform ID"].apply(lambda x: browsers[x])
data.to_csv("/Users/komlan/Project_M2/GCP_Project/doubleclick_dataset.csv")

In [29]:
import os
import pandas as pd
import json
import time
from google.cloud import pubsub_v1 #pip install --upgrade google-cloud-pubsub
credentials_path = "/Users/komlan/Project_M2/GCP_Project/noted-slice-364618-edb88fef1cbb.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
publisher = pubsub_v1.PublisherClient()
topic_path = "projects/noted-slice-364618/topics/topic-doubleclick"
#topic_path = "projects/doubleclick-project-340112/topics/topic_browser"
keys = ["Landing_Page_URL_ID","Referrer_URL","Impression_ID","Asset_ID","Active_View_Eligible_Impressions",
        "Active_View_Measurable_Impressions","Active_View_Viewable_Impressions","Activity_ID","Event_Time",
        "Event_Type","Event_Sub_Type","User_ID","Advertiser_ID","Campaign_ID","Floodlight_Configuration",
        "Ad_ID","Rendering_ID","Creative_Version","Site_ID_DCM","Placement_ID","Country_Code","State_Region",
        "Browser_Plateform_ID"]
data = pd.read_csv("/Users/komlan/Project_M2/GCP_Project/doubleclick_dataset.csv").drop(['Unnamed: 0'], axis=1)

cpt = 0
stream_data = data
print(data.shape)
for i, line in stream_data.iterrows():
    row = {}
    for j, val in enumerate(line):
        row[keys[j]] = val
    
    row["Landing_Page_URL_ID"] = str(row["Landing_Page_URL_ID"]).replace(" : ","_")
    row["User_ID"] = str(row["User_ID"])
    row["Ad_ID"] = str(row["Ad_ID"])
    row["Impression_ID"] = row["Impression_ID"].replace(" : ","_")
    row["Country_Code"] = None if pd.isnull(row["Country_Code"]) else row["Country_Code"]
    row["Event_Sub_Type"] = None if pd.isnull(row["Event_Sub_Type"]) else row["Event_Sub_Type"]
    print(row)
    attributes = json.dumps(row).encode("utf-8")
    future = publisher.publish(topic_path, attributes)
    #print(future.result())
    cpt += 1
    if cpt % 1 == 0:
        break
        time.sleep(10)

(25000, 23)
{'Landing_Page_URL_ID': 'd98ee0a4-b54d-48b4-99f8-7c25610fe885', 'Referrer_URL': 'https://www.cdiscount.com/', 'Impression_ID': 'AS983456787060', 'Asset_ID': 1011567140, 'Active_View_Eligible_Impressions': 56863856630, 'Active_View_Measurable_Impressions': 7743840, 'Active_View_Viewable_Impressions': 4436560, 'Activity_ID': 2398, 'Event_Time': '2022-01-01 05:22:29', 'Event_Type': 'CLICK', 'Event_Sub_Type': 'CLICK', 'User_ID': '7496f23a-4dfa-4942-8073-7796e18e5198', 'Advertiser_ID': 5906, 'Campaign_ID': 6768675483, 'Floodlight_Configuration': 1209785314, 'Ad_ID': 'cfefe4e2-44a8-4156-b247-7ef66853a36a', 'Rendering_ID': 565726430, 'Creative_Version': 67915, 'Site_ID_DCM': 68139, 'Placement_ID': 565726415, 'Country_Code': 'US', 'State_Region': 'United States', 'Browser_Plateform_ID': 345978365835376}


In [2]:
bq mk doubleclick_data.stream_data Landing_Page_URL_ID:STRING,Referrer_URL:STRING,Impression_ID:STRING,Asset_ID:INTEGER,Active_View_Eligible_Impressions:INTEGER,Active_View_Measurable_Impressions:INTEGER,Active_View_Viewable_Impressions:INTEGER,Activity_ID:INTEGER,Event_Time:TIMESTAMP,Event_Type:STRING,Event_Sub_Type:STRING,User_ID:STRING,Advertiser_ID:INTEGER,Campaign_ID:INTEGER,Floodlight_Configuration:INTEGER,Ad_ID:STRING,Rendering_ID:INTEGER,Creative_Version:INTEGER,Site_ID_DCM:INTEGER,Placement_ID:INTEGER,Country_Code:STRING,State_Region:STRING,Browser_Platform_ID:INTEGER
                                                                                            
                                                                                            

SyntaxError: invalid syntax (2274484936.py, line 1)

In [ ]:
{"Landing_Page_URL_ID": "573054e2-f9a0-458d-8ae2-c0a9596c13da", "Referrer_URL": "https://shopping.google.fr/", "Impression_ID": "AS983456786941", "Asset_ID": 1011567153, "Active_View_Eligible_Impressions": 56863856569, "Active_View_Measurable_Impressions": 7743725, "Active_View_Viewable_Impressions": 4436466, "Activity_ID": 2444, "Event_Time": "2022-01-01 04:38:37", "Event_Type": "CLICK", "Event_Sub_Type": NaN, "User_ID": "ef8059fd-ee67-423b-8a29-3bb9df3ddb2e", "Advertiser_ID": 5866, "Campaign_ID": 6768675418, "Floodlight_Configuration": 1209785269, "Ad_ID": "34e6ed8e-e72e-4a1c-aa7d-f2068239a5c3", "Rendering_ID": 565726392, "Creative_Version": 68215, "Site_ID_DCM": 68057, "Placement_ID": 565726431, "Country_Code": "GE", "State_Region": "Geramny", "Browser_Platform_ID": 345978365835376}




In [32]:
{"345978365835376":"Chrome","8753685264337":"Mozilla Firefox","65784523663644":"Opera","2109367534582756":"Microsoft Edge","4534637355634":"Brave"}
[0.58, 0.34, 0.0978, 0.256]

1643954280.835435

In [3]:
browser = pd.DataFrame()
browser["Browser_Plateform_ID"] = [345978365835376,8753685264337,65784523663644,2109367534582756,4534637355634]
browser["Browser_Plateform"] = ["Chrome","Mozilla Firefox","Opera","Microsoft Edge","Brave"]
browser.to_csv("/Users/komlan/Project_M2/GCP_Project/doubleclick_browser.csv", index=False)

In [110]:
{"FR":"France","GE":"Geramny","US":"United States","CA":"Canada","EN":"England","SP":"Spain","BR":"Brasil"}

['345978365835376',
 '8753685264337',
 '65784523663644',
 '2109367534582756',
 '4534637355634']

In [118]:
[0.15117245, 0.174, 0.25081, 0.116, 0.1256, 0.09, 0.08]

0.9875824499999999

In [ ]:
[0.4817245, 0.24, 0.06081, 0.156, 0.056]

In [17]:
bq mk doubleclick_data.stream_data Landing_Page_URL_ID:STRING,Referrer_URL:STRING,Impression_ID:STRING,Asset_ID:INTEGER,Active_View_Eligible_Impressions:INTEGER,Active_View_Measurable_Impressions:INTEGER,Active_View_Viewable_Impressions:INTEGER,Activity_ID:INTEGER,Event_Time:TIMESTAMP,Event_Type:STRING,Event_Sub_Type:STRING,User_ID:STRING,Advertiser_ID:INTEGER,Campaign_ID:INTEGER,Floodlight_Configuration:INTEGER,Ad_ID:STRING,Rendering_ID:INTEGER,Creative_Version:INTEGER,Site_ID_DCM:INTEGER,Placement_ID:INTEGER,Country_Code:STRING,State_Region:STRING,Browser_Platform_ID:INTEGER

,Id,Design,Price,Codeclt
0,1A,"Iphone,Faible",30,A
2,2B,"Dell,HP",34,C


'3e6cca88-7acf-4fce-95e4-7c86c2260b71'

In [2]:
bq mk doubleclick_data.doubleclick_stream Landing_Page_URL_ID:STRING,Referrer_URL:STRING,Impression_ID:STRING,Asset_ID:INTEGER,Active_View_Eligible_Impressions:INTEGER,Active_View_Measurable_Impressions:INTEGER,Active_View__Viewable_Impressions:INTEGER,Activity_ID:INTEGER,Event_Time:INTEGER,Event_Type:STRING,Event_Sub_Type:INTEGER,User_ID:STRING,Advertiser_ID:INTEGER,Campaign_ID:INTEGER,Floodlight_Configuration:INTEGER,Ad_ID:INTEGER,Rendering_ID:INTEGER,Creative_Version:INTEGER,Site_ID_DCM:INTEGER,Placement_ID:INTEGER,Country_Code:STRING,State_Region:STRING,Browser_Platform_ID:INTEGER,Browser_Platform_Version:STRING,Operating_System_ID:INTEGER,Designated_Market_Area_DMA_ID:INTEGER,City_ID:INTEGER,ZIP_Postal_Code:INTEGER,Other_Data:STRING,ORD_Value:STRING,Interaction_Time:INTEGER,Conversion_ID:INTEGER,Segment_Value_1:INTEGER,Rich_Media_Event_ID:INTEGER,Rich_Media_Event_Type_ID:INTEGER,Total_Conversions:STRING,Total_Revenue:INTEGER,Event_Timers:STRING,Event_Counters:STRING,Partner1_ID:STRING,Partner2_ID:STRING,Google_Click_ID:STRING

In [ ]:
import os
import json
import time
from google.cloud import pubsub_v1 #pip install --upgrade google-cloud-pubsub
credentials_path = "/Users/komlan/Project_M2/GCP_Project/doubleclick-project-340112-8ce2b1fd999a.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
publisher = pubsub_v1.PublisherClient()
#topic_path = "projects/doubleclick-project-340112/topics/topic_stream"
topic_path = "projects/doubleclick-project-340112/topics/topic_data_stream"
#topic_path = "projects/doubleclick-project-340112/topics/topic_browser"

keys = ["Landing_Page_URL_ID","Referrer_URL","Impression_ID","Asset_ID","Active_View_Eligible_Impressions",
"Active_View_Measurable_Impressions","Active_View__Viewable_Impressions","Activity_ID","Event_Time",
"Event_Type","Event_Sub_Type","User_ID","Advertiser_ID","Campaign_ID","Floodlight_Configuration",
"Ad_ID","Rendering_ID","Creative_Version","Site_ID_DCM","Placement_ID","Country_Code","State_Region",
"Browser_Platform_ID","Browser_Platform_Version","Operating_System_ID","Designated_Market_Area_DMA_ID",
"City_ID","ZIP_Postal_Code","Other_Data","ORD_Value","Interaction_Time","Conversion_ID","Segment_Value_1",
"Rich_Media_Event_ID","Rich_Media_Event_Type_ID","Total_Conversions","Total_Revenue","Event_Timers",
"Event_Counters","Partner1_ID","Partner2_ID","Google_Click_ID"]

stream_data = pd.read_csv("/Users/komlan/Project_M2/GCP_Project/DoubleClick_1.csv")#.drop(["Unnamed: 0"], axis=1)
stream_data = stream_data.where(pd.notnull(stream_data), None)
stream_data = stream_data[2:]
cpt = 0
for i, line in stream_data.iterrows():
    row = {}
    for j, val in enumerate(line):
        row[keys[j]] = val
    row["Landing_Page_URL_ID"] = row["Landing_Page_URL_ID"].replace(" : ","_")
    row["Impression_ID"] = row["Impression_ID"].replace(" : ","_")
    row["Country_Code"] = None if pd.isnull(row["Country_Code"]) else row["Country_Code"]
    print(row)
    attributes = json.dumps(row).encode("utf-8")

    #print(row)
    future = publisher.publish(topic_path, attributes)
    cpt += 1
    if cpt % 10 == 0:
        time.sleep(10)

In [85]:
stream_data.where(pd.notnull(stream_data), None)

,Landing Page URL ID,Referrer URL,Impression ID,Asset ID,Active View: Eligible Impressions,Active View: Measurable Impressions,Active View: Viewable Impressions,Activity ID,Event Time,Event Type,...,Segment Value 1,Rich Media Event ID,Rich Media Event Type ID,Total Conversions,Total Revenue,Event Timers,Event Counters,Partner1 ID,Partner2 ID,Google Click ID
1,ID : 1,https://www.rueducommerce.fr,impr : 3,166142,7125434,794278,1449473,5836454,1645215188833,None,...,2062568,5009463,9963164,qty=3,6032000,etm = 531894,ecn = 8951035,ykivxwbqwhucuxy240037,zpdjkkklkgupptj893180,<img src=https://fnac.com?wiwzmqjfpmonacu>


In [ ]:
import os
import json
from google.cloud import pubsub_v1 #pip install --upgrade google-cloud-pubsub
credentials_path = "/Users/komlan/Project_M2/GCP_Project/doubleclick-project-340112-8ce2b1fd999a.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
publisher = pubsub_v1.PublisherClient()
topic_path = "projects/doubleclick-project-340112/topics/topic_browser"

attributes = {
    "int64_field_0":11,
    "Browser_Plateform_ID":95323452655634,
    "Browser_Plateform": "DANTODJI"
}
attributes = json.dumps(attributes).encode("utf-8")

future = publisher.publish(topic_path, attributes)
print(future.result())

In [ ]:
import os
import json
from google.cloud import pubsub_v1 #pip install --upgrade google-cloud-pubsub
credentials_path = "/Users/komlan/Project_M2/GCP_Project/doubleclick-project-340112-8ce2b1fd999a.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
publisher = pubsub_v1.PublisherClient()
topic_path = "projects/doubleclick-project-340112/topics/topic_stream"
#topic_path = "projects/doubleclick-project-340112/topics/topic_browser"
keys = ["Landing_Page_URL_ID","Referrer_URL","Impression_ID","Asset_ID","Active_View_Eligible_Impressions",
        "Active_View_Measurable_Impressions","Active_View_Viewable_Impressions","Activity_ID","Event_Time",
        "Event_Type","Event_Sub_Type","User_ID","Advertiser_ID","Campaign_ID","Floodlight_Configuration",
        "Ad_ID","Rendering_ID","Creative_Version","Site_ID_DCM","Placement_ID","Country_Code","State_Region",
        "Browser_Platform_ID"]

stream_data = data[3:100]
for i, line in stream_data.iterrows():
    row = {}
    for j, val in enumerate(line):
        row[keys[j]] = val
    
    row["Landing_Page_URL_ID"] = str(row["Landing_Page_URL_ID"])
    row["User_ID"] = str(row["User_ID"])
    row["Ad_ID"] = str(row["Ad_ID"])
    attributes = json.dumps(row).encode("utf-8")
    print(attributes)
    break
    #future = publisher.publish(topic_path, attributes)
    #print(future.result())

In [ ]:
import os
import json
import time
from google.cloud import pubsub_v1 #pip install --upgrade google-cloud-pubsub
credentials_path = "/Users/komlan/Project_M2/GCP_Project/doubleclick-project-340112-8ce2b1fd999a.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
publisher = pubsub_v1.PublisherClient()
#topic_path = "projects/doubleclick-project-340112/topics/topic_stream"
topic_path = "projects/doubleclick-project-340112/topics/topic_data_stream"
#topic_path = "projects/doubleclick-project-340112/topics/topic_browser"
keys = ["Landing_Page_URL_ID","Referrer_URL","Impression_ID","Asset_ID","Active_View_Eligible_Impressions",
"Active_View_Measurable_Impressions","Active_View__Viewable_Impressions","Activity_ID","Event_Time",
"Event_Type","Event_Sub_Type","User_ID","Advertiser_ID","Campaign_ID","Floodlight_Configuration",
"Ad_ID","Rendering_ID","Creative_Version","Site_ID_DCM","Placement_ID","Country_Code","State_Region",
"Browser_Platform_ID","Browser_Platform_Version","Operating_System_ID","Designated_Market_Area_DMA_ID",
"City_ID","ZIP_Postal_Code","Other_Data","ORD_Value","Interaction_Time","Conversion_ID","Segment_Value_1",
"Rich_Media_Event_ID","Rich_Media_Event_Type_ID","Total_Conversions","Total_Revenue","Event_Timers",
"Event_Counters","Partner1_ID","Partner2_ID","Google_Click_ID"]

stream_data = pd.read_csv("/Users/komlan/Project_M2/GCP_Project/DoubleClick_1.csv").drop(["Unnamed: 0"], axis=1)

stream_data = stream_data[0:1]
cpt = 0
for i, line in stream_data.iterrows():
    row = {}
    for j, val in enumerate(line):
        row[keys[j]] = val
    
    attributes = json.dumps(row).encode("utf-8")
    #print(i)
    future = publisher.publish(topic_path, attributes)
    print(future.result())
    """cpt += 1
    if cpt % 2 == 0:
        time.sleep(3)"""

In [26]:
import os
import pandas as pd
import json
import time
from google.cloud import pubsub_v1 #pip install --upgrade google-cloud-pubsub
credentials_path = "/Users/komlan/Project_M2/GCP_Project/doubleclick-364815-8e1a01a8d071.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
publisher = pubsub_v1.PublisherClient()

In [42]:
import os
import pandas as pd
import json
import time
from google.cloud import pubsub_v1 #pip install --upgrade google-cloud-pubsub
credentials_path = "/Users/komlan/Project_M2/GCP_Project/doubleclick-364815-8e1a01a8d071.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
publisher = pubsub_v1.PublisherClient()
topic_path = "projects/doubleclick-364815/topics/topic_doubleclick"
keys = ["Landing_Page_URL_ID","Referrer_URL","Impression_ID","Asset_ID","Active_View_Eligible_Impressions",
        "Active_View_Measurable_Impressions","Active_View_Viewable_Impressions","Activity_ID","Event_Time",
        "Event_Type","Event_Sub_Type","User_ID","Advertiser_ID","Campaign_ID","Floodlight_Configuration",
        "Ad_ID","Rendering_ID","Creative_Version","Site_ID_DCM","Placement_ID","Country_Code","State_Region",
        "Browser_Plateform_ID"]
data = pd.read_csv("/Users/komlan/Project_M2/GCP_Project/doubleclick_dataset.csv").drop(['Unnamed: 0'], axis=1)

cpt = 0
stream_data = data
print(data.shape)
for i, line in stream_data.iterrows():
    row = {}
    for j, val in enumerate(line):
        row[keys[j]] = val
    
    row["Landing_Page_URL_ID"] = str(row["Landing_Page_URL_ID"]).replace(" : ","_")
    row["User_ID"] = str(row["User_ID"])
    row["Ad_ID"] = str(row["Ad_ID"])
    row["Impression_ID"] = row["Impression_ID"].replace(" : ","_")
    row["Country_Code"] = None if pd.isnull(row["Country_Code"]) else row["Country_Code"]
    row["Event_Sub_Type"] = None if pd.isnull(row["Event_Sub_Type"]) else row["Event_Sub_Type"]
    attributes = json.dumps(row).encode("utf-8")
    future = publisher.publish(topic_path, attributes)
    cpt += 1
    if cpt % 10 == 0:
        time.sleep(30)

(25000, 23)


KeyboardInterrupt: 

In [57]:
import hashlib
import pandas as pd
import itertools
from itertools import combinations, product

name_pass = 5
ani_pass = 0
min_pass =0
max_pass = 7
#open dictionary
with open('/Users/komlan/Downloads/projetJTR/dico_animaux.txt', 'r', encoding='utf-8') as f:
    dictionary = f.readlines()
dictionary = [el.replace("\n",'')  for el in dictionary]

#open xlsx
users = pd.read_csv('/Users/komlan/Downloads/projetJTR/Liste_Atrouver.csv', sep=";")
users = users[["Prénom","Nom","Mot de passe cryptés","Mot de passe en clair"]]
users
for i, user in users.iterrows():
    for animal in dictionary:
        combi = (user['Nom'][:name_pass]+animal[:ani_pass]+'123')
        darray = []
        for k in range(min_pass, max_pass):
            darray.extend(list(itertools.permutations(combi, k)))
        darray = [''.join(el) for el in darray]
        #print(darray)
        for da in darray:
            result = hashlib.md5(da.encode()).hexdigest()
            #print(da,': ', '=>',chiff,': ',user['Mot de passe cryptés'])
            if result in user['Mot de passe cryptés']:
                print('trouvé mot de pass:', da)


In [41]:
lis = [['ud','3dfe3d7a171097592cb00d6a57b6f845'],
       ['gvhj','02000c8c8ee02c8f409c5a4cb9d5219c'],
       ['fvghs','e395ee799c3ec879c24c2bebf9a03a60'],
       ['hvdfb','ec316ce1fabf0005553fbe9666bf9496'],
       ['hjd','d928472364a98d5e1b13839145d3836b']]

pas = 'fvghs'
for el in lis:
    if el[1] == hashlib.md5(pas.encode()).hexdigest():
        print('trouve pass: ', el[0])

trouve pass:  fvghs


In [62]:
!pip install aspose-words
#from docx2pdf import convert

#convert("/Users/komlan/Downloads/test_doc.docx", "/Users/komlan/Downloads/test_doc.pdf")


ERROR: Could not find a version that satisfies the requirement aspose-words (from versions: none)
ERROR: No matching distribution found for aspose-words


In [7]:
import img2pdf
from PIL import Image

files = ["/Users/komlan/Enovsky/logo_enovsky.png","filedata.docx",
         "filejh.sjg.pdf","icdh"]
for file_path in files:
    filename = file_path.split('/')[-1]
    extention = filename.split('.')[-1]
    if extention in ["png", "jpeg","jpg"]:
        image = Image.open(file_path)
        pdf_bytes = img2pdf.convert(image.filename)
        file = open("/Users/komlan/Downloads/"+filename.replace(extention, "pdf"), "wb")
        file.write(pdf_bytes)
        image.close()
        file.close()
    elif extention in ["docx"]:
        print(file,"=>docx")
    elif extention in ["pdf"]:
        print(file,"=>pdf")
    else:
        print("bad format")

bad format
bad format
bad format
bad format


In [55]:
import requests
import json
docx_url = "https://api.edu.enovsky.com/public/tmp/51369dd07caf4400b352c9dc9b9b8760/input.docx"
url = 'https://edu.enovsky.com/convert_to_pdf';
params = {
        'origin': 'extern',
        'file_url':  docx_url,
        'id': 0
        }
print(json.dumps(params))
resp = requests.post('https://edu.enovsky.com/convert_to_pdf', data=params)

open('/Users/komlan/Downloads/facebook.pdf', 'wb').write(resp.content)


{"origin": "extern", "file_url": "https://api.edu.enovsky.com/public/tmp/51369dd07caf4400b352c9dc9b9b8760/input.docx", "id": 0}


120838

In [413]:
path = "JE/ME/DONNE/AW/TOI"
path_rec = []
for el in path.split('/'):
    path_rec.append(el)
    directory = '/'.join(path_rec)
    try:
        nextcloud.mkdir(directory)
    except:
        continue

In [ ]:
remote_path = 'TGO/UL/FDS/raw/file.pdf'


In [404]:
import nextcloud_client as nc
nextcloud = nc.Client('https://cloud.enovsky.com')
nextcloud.login('admin', 'nextcloud00')
source_path = '/Users/komlan/Downloads/facebook.pdf'
target_path = 'WAREHOUSE/RAW_WAREHOUSE/TGO/UL/FDS/facebook.pdf'
help(nextcloud)
nextcloud.mkdir('testdir/el/gh')

Help on Client in module nextcloud_client.nextcloud_client object:

class Client(builtins.object)
 |  Client(url, **kwargs)
 |  
 |  nextCloud client
 |  
 |  Methods defined here:
 |  
 |  __init__(self, url, **kwargs)
 |      Instantiates a client
 |      
 |      :param url: URL of the target nextCloud instance
 |      :param verify_certs: True (default) to verify SSL certificates, False otherwise
 |      :param dav_endpoint_version: None (default) to force using a specific endpoint version
 |      instead of relying on capabilities
 |      :param debug: set to True to print debugging messages to stdout, defaults to False
 |  
 |  accept_remote_share(self, share_id)
 |      Accepts a remote share
 |      
 |      :param share_id: Share ID (int)
 |      :returns: True if the operation succeeded, False otherwise
 |      :raises: HTTPResponseError in case an HTTP error status was returned
 |  
 |  add_user_to_group(self, user_name, group_name)
 |      Adds a user to a group.
 |      
 

HTTPResponseError: HTTP error: 405

In [375]:
filePath = 'TGO/UL/FDS/tgo_ul_fds_6.pdf'
filePath = "/WAREHOUSE/RAW_WAREHOUSE/"+filePath;
params = {
    'table': 'get_sftp_file',
    'file_path': filePath
}
resp = requests.post('https://edu.enovsky.com/request_online_topics', data=params)
open('/Users/komlan/Downloads/file_download.pdf', 'wb').write(resp.content)

152007

In [377]:
import os
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import time
root_path = '/Users/komlan/Downloads/test'

options = Options()
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.dir", root_path+'/outputs/')
browser = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)

browser.implicitly_wait(5) # seconds

[WDM] - Downloading: 19.0kB [00:00, 4.83MB/s]                                                                      
[WDM] - Downloading: 100%|████████████████████████████████████████████████████| 1.83M/1.83M [00:00<00:00, 2.65MB/s]
/var/folders/8t/83qwqzx52cs5gmzbyklq6v_w0000gn/T/ipykernel_65127/380439365.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)


WebDriverException: Message: Service /Users/komlan/.wdm/drivers/geckodriver/macos-aarch64/0.32/geckodriver unexpectedly exited. Status code was: -9


In [372]:
#Downloads
shutil.rmtree(root_path+'/outputs/')
os.mkdir(root_path+'/outputs/')
outputs = browser.find_elements(By.XPATH, "//li[@class='ant-menu-item font-ibm-plex-sans text-sm pdf-item ']")

for output in outputs:
    option = output.find_element(By.CLASS_NAME, 'item-button')
    option.click()
    docx_element = browser.find_element(By.XPATH, "//ul[@class='popover-content']").find_elements(By.XPATH, "//li[@class='w-full flex justify-between popover-content-item w-40 ']")[2]
    docx_element.click()
    time.sleep(1)

/var/folders/8t/83qwqzx52cs5gmzbyklq6v_w0000gn/T/ipykernel_65127/2905521807.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)


WebDriverException: Message: Service /Users/komlan/.wdm/drivers/geckodriver/macos-aarch64/0.32/geckodriver unexpectedly exited. Status code was: -9


In [349]:
browser.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div/div[3]/button").click()


#Upload files
files = os.listdir(root_path)
try:
    os.mkdir(root_path+'/outputs/')
except:
    ""
    
for file_name in files:
    upload_file = browser.find_element(By.XPATH, "//input[@type='file' and @accept='application/pdf']")
    #upload_file.send_keys(root_path+'/'+file_name)

container = browser.find_element(By.XPATH, "//ul[@id='pdfs$Menu' and @class='ant-menu ant-menu-sub ant-menu-inline']")
outputs = browser.find_elements(By.XPATH, "//li[@class='ant-menu-item font-ibm-plex-sans text-sm pdf-item ']")

for output in outputs:
    option = output.find_element(By.CLASS_NAME, 'item-button')
    option.click()
    docx_element = browser.find_element(By.XPATH, "//ul[@class='popover-content']").find_elements(By.XPATH, "//li[@class='w-full flex justify-between popover-content-item w-40 ']")[2]
    docx_element.click()

<selenium.webdriver.remote.webelement.WebElement (session="de86c81f-25fd-4905-a74f-7617919e35af", element="1e93ca58-3dec-4bc5-ae70-a481fb184219")>

In [351]:
!pip install tk

In [370]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
agent = f"user-agent={user_agent}"
agent

'user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'

In [384]:
import os
root_path = '/Users/komlan/Downloads/test'
os.path.exists(root_path)
#os.mkdir(root_path)
os.listdir(root_path)

['.DS_Store', 'new.pdf', 'outputs']

In [387]:
file_name = 'tgo_ul_fds_ev329ky_raw.pdf'
'/'.join([el.upper() for el in file_name.split('_')[:3]])

'TGO/UL/FDS'

In [393]:
from datetime import datetime
my_date = "2022-07-19 15:25:25.686028"
 
# convert datetime string into date,month,day and
# hours:minutes:and seconds format using strptime
d = datetime.strptime(my_date, "%d-%b-%Y-%H:%M:%S")
print(d)

ValueError: time data '2022-07-19 15:25:25.686028' does not match format '%d-%b-%Y-%H:%M:%S'

In [363]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

server_url = "https://edu.enovsky.com"
dc = DesiredCapabilities.HTMLUNIT
wd = webdriver.Remote(server_url, dc)
wd.get('http://www.google.com')

/var/folders/8t/83qwqzx52cs5gmzbyklq6v_w0000gn/T/ipykernel_65127/1704379198.py:6: DeprecationWarning: desired_capabilities has been deprecated, please pass in an Options object with options kwarg
  wd = webdriver.Remote(server_url, dc)


WebDriverException: Message: <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<title>Error</title>
</head>
<body>
<pre>Cannot POST /session</pre>
</body>
</html>



In [369]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"

options = webdriver.ChromeOptions()
options.headless = True
options.add_argument(f'user-agent={user_agent}')
options.add_argument("--window-size=1920,1080")
options.add_argument('--ignore-certificate-errors')
options.add_argument('--allow-running-insecure-content')
options.add_argument("--disable-extensions")
options.add_argument("--proxy-server='direct://'")
options.add_argument("--proxy-bypass-list=*")
options.add_argument("--start-maximized")
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(executable_path='/Users/komlan/Downloads/chromedriver_mac_arm64/chromedriver', options=options)


/var/folders/8t/83qwqzx52cs5gmzbyklq6v_w0000gn/T/ipykernel_65127/4241056066.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/komlan/Downloads/chromedriver_mac_arm64/chromedriver', options=options)


WebDriverException: Message: Service /Users/komlan/Downloads/chromedriver_mac_arm64/chromedriver unexpectedly exited. Status code was: -9


output.pdf

facebook.pdf

tgo_ul_fds_ev319ky_raw.pdf

math_12.pdf

sagvfVpk.pdf

Untitled.pdf



In [ ]:
for output in outputs:
    option = output.find_element(By.XPATH, "//button[@class='item-button']")
    file_name = output.find_element(By.XPATH, "//div[@class='text-ellipsis whitespace-nowrap overflow-hidden']")#.text()
    
    #ActionChains(output).move_to_element(option).click().perform()
    print(file_name.text)
    """try:
        option.click()
    except:
        """""
    
    """convert_panel = browser.find_element(By.XPATH, "//ul[@class='popover-content']")
    docx_butt = convert_panel.find_elements(By.XPATH, "//li[@class='w-full flex justify-between popover-content-item w-40 ']")[2]
    docx_butt.click()"""

new.pdf
new.pdf
new.pdf
new.pdf
new.pdf
new.pdf
new.pdf


In [ ]:
#Delete old files

In [ ]:
email = 
print(email.text)
email.send_keys(user_email)
password = browser.find_element_by_class_name('df-card__main')
password.send_keys(user_password)

button = browser.find_element_by_class_name('btn_l1_quaternary')
button.click()